In [ ]:
import pandas as pd
import numpy as np
import pyarrow as pa


In [ ]:
df_uncompleted = pd.read_parquet('uncompleted.parquet',engine='pyarrow')
df_uncompleted = df_uncompleted[:len(df_uncompleted)//2]
df_completed  = pd.read_parquet('merged_complete.parquet',engine='pyarrow')

In [ ]:
completeA = df_uncompleted[df_uncompleted['rawText'].notnull()]
incompleteA = df_uncompleted[df_uncompleted['rawText'].isnull()]

In [ ]:
incompleteB  = pd.read_parquet('merged_incomplete.parquet',engine='pyarrow')

In [ ]:
merged_complete = pd.concat([completeA, df_completed], ignore_index=True)
merged_incomplete = pd.concat([incompleteA, incompleteB], ignore_index=True)

In [ ]:
def mergeProgress():
 # Load the two Parquet files
    df1 = pd.read_parquet('completed.parquet')
    df2 = pd.read_parquet('storage/merged_complete.parquet')
    merged_complete = pd.concat([df1, df2], ignore_index=True)
    merged_complete.to_parquet('storage/merged_complete.parquet', index=False)
    print(f"{len(merged_complete)}")
    # Decompose: Filter rows where 'rawText' is not null
    completeA = df1[df1['rawText'].notnull()]
    completeB = df2[df2['rawText'].notnull()]
    # Decompose: Filter rows where 'rawText' is null
    incompleteA = df1[df1['rawText'].isnull()]
    incompleteB = df2[df2['rawText'].isnull()]
    # Merge the dataframes: nulls with nulls, not nulls with not nulls
    merged_complete = pd.concat([completeA, completeB], ignore_index=True)
    merged_incomplete = pd.concat([incompleteA, incompleteB], ignore_index=True)
    # Optional: Save the merged dataframes to Parquet (if needed)
    merged_complete.to_parquet('storage/merged_complete.parquet', index=False)
    merged_incomplete.to_parquet('storage/merged_incomplete.parquet', index=False)

In [ ]:
merged_complete.to_parquet('complete.parquet', index=False)
merged_incomplete.to_parquet('incomplete.parquet', index=False)

In [ ]:
df_merged_dept = pd.read_parquet('completed.parquet',engine='pyarrow')
df_merged_dept2 = pd.read_parquet('completed_2.parquet',engine='pyarrow')
df_nonMissingdept = df_merged_dept[df_merged_dept['department']!='MISSING']
df_Missingdept = df_merged_dept[df_merged_dept['department']=='MISSING']
#in the preview just select 'rawText' and 'department' columns
df_filter = df_Missingdept[['id_text','rawText','department']]
pd.set_option('display.max_colwidth', None)
df_filter


In [ ]:
#Just print the raw text where id_text is something
(df_filter[df_filter['id_text'] == 'A Abanov texas a and m university'])['rawText'].values[0].split('\n')


In [ ]:
from dess.nlp import populate_faculty_columns

In [ ]:
x = df_Missingdept.head(1)['rawText'].values[0]
print(f'{x[0]}\n\n {x[1]} \n\n {x[2]} \n\n {x[3]} \n\n')

In [ ]:
df_merged_dept2 = pd.read_parquet('completed_2.parquet',engine='pyarrow')
df_merged_dept2_filter = df_merged_dept2[(df_merged_dept2['isProfessor']==True)]
df_merged_dept2_nonmissing = df_merged_dept2[(df_merged_dept2['isProfessor']==True) & (df_merged_dept2['department']!='MISSING')]
pd.set_option('display.max_colwidth', None)
df_merged_dept2_nonmissing

In [ ]:
df_merged_dept_nlp = pd.read_parquet('completed_nlp.parquet',engine='pyarrow')
df_merged_dept_nlp_filter = df_merged_dept_nlp[(df_merged_dept_nlp['isProfessor']==True)]
df_merged_dept_nlp_nonmissing = df_merged_dept_nlp[(df_merged_dept_nlp['department']!='MISSING')]
pd.set_option('display.max_colwidth', None)
df_merged_dept_nlp_nonmissing

In [ ]:
df_new = pd.read_parquet('completed.parquet',engine='pyarrow')
df_new_filter = df_new[(df_new['department']=='MISSING')& (df_new['isProfessor']==True)]
old = len(df_new_filter)
df_new_filter['department'] = df_new_filter['rawText'].apply(lambda x: extract_department_regex(x))
df_new_new_filter = df_new_filter[(df_new_filter['department']!='MISSING')]
new = len(df_new_new_filter)
# df_new_filter['rawText'].values[0].tolist()
print(old, new)



In [ ]:
import re
def extract_department_regex(rawText):
    primary_patterns = [
        r'professor of ([A-Za-z]+),',              # Professor of + first word
        r'department of ([A-Za-z]+)',             # Department of + first word
        r'professor in the ([A-Za-z]+)'           # Professor in the + first word
    ]

    backup_patterns = [
        r'school of ([A-Za-z]+)',                 # School of + first word
        r'college of ([A-Za-z]+)',                # College of + first word
        r'book on ([A-Za-z]+)',                   # Book on + first word
        r'in the area of ([A-Za-z]+)',            # In the area of + first word
        r'research primarily focused on ([A-Za-z]+)'  # Research primarily focused on + first word
    ]

    if rawText is None: return "MISSING"
    
    #concat_str_lst = [" ".join(rawText)]
    # Iterating over snapshots and trying primary patterns
    for text in rawText:
        # print(text)
        # print("\n")
        for pattern in primary_patterns:
            #print(pattern,text)
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                return match.group(1).strip()

    # Iterating over snapshots and trying backup patterns
    for text in rawText:
        for pattern in backup_patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                return match.group(1).strip()

    return "MISSING" 

In [ ]:
extract_department_regex(df_new_filter['rawText'].values[0])

In [ ]:
df_new_new_filter

In [104]:
df_merged_dept_search = pd.read_parquet('completed_search.parquet',engine='pyarrow')
df_merged_dept_search_filter = df_merged_dept_search[(df_merged_dept_search['isProfessor']==True)]
df_merged_dept2_nonmissing = df_merged_dept_search[(df_merged_dept_search['isProfessor']==True) & (df_merged_dept_search['department']!='MISSING')]
pd.set_option('display.max_colwidth', None)
print(len(df_merged_dept_search),len(df_merged_dept))

36091 36091


In [ ]:
df_merged_dept2_nonmissing.head(70)

In [103]:
pd.set_option('display.max_colwidth', None)
df_mdip = df_merged_dept_search[(df_merged_dept_search['isProfessor']==True) & (df_merged_dept_search['department']=='MISSING')][:51]
df_filtermdip = df_mdip[['fullid','id_text','rawText',]]
df_filtermdip
# df_filtermdip.to_csv('debugging.csv',index=False)

,fullid,id_text,rawText
1,4.0,A Abbassi texas a and m university-commerce,"[Amir Abbassi, Associate Professor, Counseling. Contact Amir: email Amir.Abbassi@tamuc.edu, phone 903.886.5594, Office Binnion Hall 204., Known as a great educator, mentor and friend, Dr. Abbassi spent a large part of his career in academia, training the next generation to show up with kindness, ..., Professor at Texas A&M University-Commerce · Experience: Texas A&M University-Commerce · Location: Richardson · 465 connections on LinkedIn., Amir ABBASSI | Cited by 27 | of Texas A&M University-Commerce, Texas | Read 4 publications | Contact Amir ABBASSI.]"
6,10.0,A Acevedo sul ross rio grande college,"[Assistant Professor, Sul Ross State University Rio Grande College. Jan 2016 - Present 8 years 8 months. Texas Tech University Graphic. Spanish Instructor., May 7, 2019 — Grant recipients were: Betsy Evans, Education and Outreach, Library; Dr. Ana Acevedo, Ilda Gonzalez and Dr. Filemon Zamora, B.A. Spanish; Dr., 328 pages, Missing: Acevedo | Show results with: Acevedo]"
20,24.0,A Aharireddi university of california,"[A Hari Reddi's profile, publications, research topics, and co-authors., A. Hari Reddi (born October 20, 1942) is a University of California Distinguished Professor and holder of the Lawrence J. Ellison Endowed Chair in ..., , ]"
21,25.0,A Ahmed texas a and m university,"[Mar 1, 2024 — Biography. Professor Ahmed graduated from the University of Rochester in 1992 with a PhD in Accounting and Finance., She holds a Bachelor's degree in Communication Studies and Philosophy (2016), a Masters in Communication (2021), and is now pursuing a doctoral degree at Texas ..., Anwer Ahmed. Texas A & M University. Verified email at mays.tamu.edu. Financial AccountingGovernanceFundamental AnalysisEarnings Management., Sep 1, 2023 — Aden Ahmed, Interim Chair & Associate Professor, Ph.D. Portland State University, Oregon. Research Interests: Featured Research and Selected Publications.]"
36,40.0,A Alcantara university of houston,"[Associate Professor at University of Houston · Experience: University of Houston · Location: Houston. View Adriana Alcantara's profile on LinkedIn, ..., Jul 28, 2016 — Plaintiff alleges that she was harassed on the basis of her sex, race, and national origin, by Dr. Leigh Leasure, a younger female professor., Dr. Alcantara is a pretty good professor- pay attention to lectures and use the book for study. The exams aren't all that difficult but the final is ..., United States District Court Southern District of Texas. Circuit. 5th. Office Location. Houston. Case Type. civil. Nature of Suit. Civil Rights Employment ...]"
41,45.0,A Allen texas a and m university,"[Sep 14, 2023 — Natalie Allen joined the Texas A&M University faculty as a Lecturer in 1989 and was promoted to Senior Lecturer in 1996., Dr. Amanda Allen is an associate professor at Texas A&M University-Central Texas in Killeen, Texas. She earned her EdD in Special Education at the ..., Instructional Associate Professor ; Blocker 348 · +1 979 862 4190 · angieallen <at> tamu.edu · https://people.tamu.edu/~angieallen/ · B.S. Texas A&M University, 2001, Alex Allen - Height 6-2, Weight 195, Position WR, Class Junior, Hometown Daingerfield, Texas, High School Daingerfield HS.]"
56,61.0,A Anazligundes university of california,"[Professor A. Nazli Gündeş. Electrical and Computer Engineering. University of California; Davis, CA 95616. Email: angundes at ucdavis.edu; Phone: Privacy and ..., Professor A. Nazli Gündeş. Electrical and Computer Engineering. University of California; Davis, CA 95616. Email: angundes at ucdavis.edu; Phone: Privacy and ..., A. N. Gündeş received the B.S., M.S., and Ph.D. degrees in electrical engineering and computer science from the University of California at Berkeley, Berkeley, ..., Apr 25, 2024 — List of computer science publications by A. Nazli Gündes.]"
77,84.0,A Arenson university of texas at el paso,"[Dr. Arenson is an amazingly good professor and an even 

In [107]:
df_complete = pd.read_parquet('complete.parquet',engine='pyarrow')
df_complete

,fullid,id_text,id_name,id_university,isProfessor,rawText,department
0,8.0,A Abubaker virginia commonwealth university,A Abubaker,virginia commonwealth university,None,"[A. Abubaker Professor Chair of Oral and Maxillofacial Surgery, Oral and Maxillofacial Surgery, Dental Building 1, Phone: (804) 828-0602, Email: abubaker@vcu.edu, (804) 628-6637 Request an Appointment Specialty Oral and Maxillofacial Department Surgery Locations Nelson Clinic 401 N. 11th Street Richmond, VA 23219, 1 page, A. Omar Abubaker. Virginia Commonwealth University | VCU · Department of Oral and Maxillofacial Surgery.]",
1,48.0,A Alofsin university of texas at austin,A Alofsin,university of texas at austin,None,"[Professor Emeritus · Anthony Alofsin has taught a broad range of courses from advanced design to histories of modernism to global surveys of architecture over ..., 2013, Award-winning architect, author, exhibition curator, and teacher, Dr. Alofsin is internationally recognized as one of the world's leading authorities on the ..., In 2017 he began donating material to establish the Anthony Alofsin Archive at the University of Texas in Austin. The vast collection contains research ...]",
2,111.0,A Babbili texas a and m university - corpus christi,A Babbili,texas a and m university - corpus christi,None,"[Specializing in media studies, global communications, upper educational management, innovative trends in international communication and management. Professor., Jul 25, 2023 — Provost and VP Academic Affairs for Texas A&M University Corpus Christi beginning 2007, now retired from academia; Brings nearly 35 years of ..., Anantha Babbili is a professor in the Communication department at Texas A&M University at Corpus Christi - see what their students are saying about them or ..., Babbili, Anantha S. Retired - Adjunct Faculty (361) 563-5177 (Office). Institution. Texas A&M University - Corpus Christi. Department. Retired - College of ...]",
3,146.0,A Barry texas a and m university,A Barry,texas a and m university,None,"[Dr. Barry is a health behavior social scientist, with specific training and expertise in alcohol use, alcohol-induced impairment, and intoxication., Adam Barry is a professor and department head for the Department of Health Behavior at the Texas A&M University School of Public Health. Barry is a health ..., Adam Barry is a professor in the Health Physical Education department at Texas A&M University at College Station - see what their students are saying about ..., I'm a health behavior social scientist, with specific training and expertise in alcohol use, alcohol-induced impairment, and intoxication.]",
4,164.0,A Beckman university of illinois at urbana-champaign,A Beckman,university of illinois at urbana-champaign,None,"[Directory · History of the Beckman Institute · Job opportunities and human... · News, The Beckman Institute was the first interdisciplinary research institute on the University of Illinois campus; and a model across the world for creating ..., The Beckman Institute for Advanced Science and Technology is a unit of the University of Illinois Urbana-Champaign dedicated to interdisciplinary research., Feedback]",
...,...,...,...,...,...,...,...
172927,482828.0,Umit Catalyurek georgia institute of technology,Umit Catalyurek,georgia institute of technology,None,"[2000, Research interests: Combinatorial Scientific Computing · See more, See more, Richard Vuduc]",
172928,491903.0,Wasim Maziak florida international university,Wasim Maziak,florida international university,None,"[66, Research interests: Tobacco Control, Emerging Tobacco Products, Cardiovascular Health, See more, Raed Bahelah]",
172929,491971.0,Waugh Waugh university of california,Waugh Waugh,university of california,None,"[Nov 30, 2021 — Professor Joan Waugh of the UCLA History Department researches and writes about nineteenth-century America, specializing in the Civil War, Reconstruction, and ..., 1988, Nov 30, 2021 — Scott Waugh ... Professor Emeritus and For

In [111]:
pd.set_option('display.max_colwidth', None)
df_upload = pd.read_parquet('upload.parquet',engine='pyarrow')
df_mdip = df_upload[(df_upload['isProfessor']==True) & (df_upload['department']!='MISSING')]
df_filtermdip = df_mdip[['fullid','id_text','rawText',]]
df_mdip

,fullid,id_text,id_name,id_university,isProfessor,rawText,department
0,8.0,A Abubaker virginia commonwealth university,A Abubaker,virginia commonwealth university,True,"[A. Abubaker Professor Chair of Oral and Maxillofacial Surgery, Oral and Maxillofacial Surgery, Dental Building 1, Phone: (804) 828-0602, Email: abubaker@vcu.edu, (804) 628-6637 Request an Appointment Specialty Oral and Maxillofacial Department Surgery Locations Nelson Clinic 401 N. 11th Street Richmond, VA 23219, 1 page, A. Omar Abubaker. Virginia Commonwealth University | VCU · Department of Oral and Maxillofacial Surgery.]",Oral
2,111.0,A Babbili texas a and m university - corpus christi,A Babbili,texas a and m university - corpus christi,True,"[Specializing in media studies, global communications, upper educational management, innovative trends in international communication and management. Professor., Jul 25, 2023 — Provost and VP Academic Affairs for Texas A&M University Corpus Christi beginning 2007, now retired from academia; Brings nearly 35 years of ..., Anantha Babbili is a professor in the Communication department at Texas A&M University at Corpus Christi - see what their students are saying about them or ..., Babbili, Anantha S. Retired - Adjunct Faculty (361) 563-5177 (Office). Institution. Texas A&M University - Corpus Christi. Department. Retired - College of ...]",Communication
3,146.0,A Barry texas a and m university,A Barry,texas a and m university,True,"[Dr. Barry is a health behavior social scientist, with specific training and expertise in alcohol use, alcohol-induced impairment, and intoxication., Adam Barry is a professor and department head for the Department of Health Behavior at the Texas A&M University School of Public Health. Barry is a health ..., Adam Barry is a professor in the Health Physical Education department at Texas A&M University at College Station - see what their students are saying about ..., I'm a health behavior social scientist, with specific training and expertise in alcohol use, alcohol-induced impairment, and intoxication.]",Health
5,169.0,A Belarbi university of houston,A Belarbi,university of houston,True,"[Dr. Abdeldjelil Belarbi, Ph.D., PE (TX, MO), F.ACI, F.SEI, F.ASCE, F.IIFC Hugh Roy and Lillie Cranz Cullen Distinguished Professor, Dr. Abdeldjelil Belarbi, Professor, Department of Civil and Environmental Engineering, Cullen College of Engineering., Distinguished Professor at University of Houston Engineering · Abdeldjelil ""DJ"" Belarbi is an Algerian-American Structural Engineer and Researcher whose ..., Belarbi has a 5% bonus for participation. He is a very sarcastic professor and gets upset by weird things, but does his best to make class interesting. Exams ...]",Civil
6,187.0,A Bernal university of texas at el paso,A Bernal,university of texas at el paso,True,"[Ricardo A. Bernal, Associate Professor, Chemistry and Biochemistry, Bioinformatics, Environmental Science and Engineering, Ricardo A. Bernal. The University of Texas at El Paso. Verified email at utep.edu - Homepage · cryo-Electron MicroscopyX-ray crystallography ..., 17 pages, Ricardo Andres Bernal currently works at the Department of Chemistry, University of Texas at El Paso. Ricardo does research in Structural Biology, Virology and ...]",Chemistry
...,...,...,...,...,...,...,...
172920,465786.0,Tarek Abichou florida state university,Tarek Abichou,florida state university,True,"[Dr. Tarek Abichou, Professor, CEE - Civil & Environmental Engineering, Phone Numbers (850) 410-6661, Office Sliger Building, Room 274, abichou@eng.famu.fsu.edu, Florida State University - ‪‪Cited by 4300‬‬ - ‪Civil and Environmental Engineering‬ - ‪Geotechnical Engineering‬ - ‪Geo-environmental Engineering‬ ..., Recent Interests: Geoenvironmental Engineering; Geotechnical Engineering; Sustainable Soild Waste Management; Measuring, Modeling, and Mitigating Fugitive ..., Tarek Abichou, FAMU-FSU College of Engineering, University of Wisconsin-Madison, About, As a Professo